# Lending Club Data Loan Default Prediction

In [2]:
library(dplyr)

library(lubridate)
library(ggplot2)
library(dplyr)
library(stringr)
library(caret)
library(rpart)
#library(rattle)
library(ROSE)
library(ROCR)
library(MASS)
library(ipred)
library(plyr)
library(rpart.plot)
require(gridExtra) # also loads grid

library(data.table)
options(datatable.print.nrows = 500)
options(width = 110)
options(repr.plot.width=11, repr.plot.height=5)

In [3]:
# Set working path:
wkdir = "/Users/qiaolinchen/Documents/LendingClub/"
raw_data_path = paste0(wkdir, 'raw_data/')

## 1.Load data from CSV files

In [ ]:
loans= read.csv(df_2016_2017, file= paste0(wkdir ,'loan_2016-2017.csv') )